# 1. Import required libraries

Install optional packages if missing (yfinance used for convenience)

In [ ]:
import subprocess
import sys
from pathlib import Path

# Install required packages from pyproject.toml dependencies
required = ["pandas", "numpy", "matplotlib", "yfinance"]
for pkg in required:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker

# Add model package to path (works for shared/cloned repos)
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

%matplotlib inline

## API Configuration

Set your API keys below to enable all features. This notebook uses Anthropic (Claude) and Perplexity AI.

In [ ]:
# ===== CONFIGURE YOUR API KEYS BELOW =====
# Replace the placeholder values with your actual API keys
# These will be set as environment variables for the notebook session

# Get your keys from:
# - Anthropic: https://console.anthropic.com/
# - Perplexity: https://www.perplexity.ai/settings/api

ANTHROPIC_API_KEY = ""  # sk-ant-...
PERPLEXITY_API_KEY = ""  # pplx-...
FRED_API_KEY = ""  # (Optional, for FRED economic data)

# Set environment variables
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
os.environ["PERPLEXITY_API_KEY"] = PERPLEXITY_API_KEY
if FRED_API_KEY:
    os.environ["FRED_API_KEY"] = FRED_API_KEY

# Verify keys are set
print("API Configuration:")
print(f"  Anthropic: {'✓ Set' if os.getenv('ANTHROPIC_API_KEY') else '✗ Not set'}")
print(f"  Perplexity: {'✓ Set' if os.getenv('PERPLEXITY_API_KEY') else '✗ Not set'}")
print(f"  FRED: {'✓ Set' if os.getenv('FRED_API_KEY') else '(Optional)'}")

# Stage 1: Stock Analytics

In [ ]:
from model.orchestration.stock_analytics_orchestrator import StockAnalyticsOrchestrator

In [ ]:
# Instantiate orchestrator for SNOW
TICKER = "SNOW"
orchestrator = StockAnalyticsOrchestrator(TICKER)

## get_technical_metrics
Retrieve technical metrics from the `StockAnalyticsOrchestrator` and print selected values.


In [ ]:
tech = orchestrator.get_technical_metrics()
hist = orchestrator.stock_data_provider.history()

print(f"Technical Metrics for {TICKER}")
print(f"Current Price: ${tech.get('current_price'):.2f}")
print(f"52-Week High: ${tech.get('price_252d_high'):.2f}")
print(f"52-Week Low: ${tech.get('price_252d_low'):.2f}")
print(f"Historical Volatility: {tech.get('historical_volatility_annual'):.2%}")
print(f"Bollinger Position: {tech.get('bollinger_position_20d_2std'):.2f}")
print(f"\nData points available: {len(hist)}")

In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 6))

# Price and SMA
ax1.plot(hist.index, hist['Close'], label='Close Price', linewidth=2, color='#1f77b4')
ax1.plot(hist.index, hist['Close'].rolling(20).mean(), label='20-Day SMA', linewidth=2, color='#ff7f0e', alpha=0.8)
ax1.set_ylabel('Price ($)', fontsize=11)
ax1.tick_params(axis='y')
ax1.grid(True, alpha=0.3)

# Volume on secondary axis
ax2 = ax1.twinx()
ax2.fill_between(hist.index, hist['Volume'], alpha=0.2, color='gray', label='Volume')
ax2.set_ylabel('Volume', fontsize=11)
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f'{x/1e6:.0f}M'))

# Format x-axis as dates
ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
fig.autofmt_xdate(rotation=45)

ax1.set_title(f'{TICKER} - Price, SMA 20, and Volume', fontsize=13, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax2.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()

## Financial Metrics
Retrieve financial metrics and show primary accounting values.


In [ ]:
fin = orchestrator.get_financial_metrics()
income_df = orchestrator.stock_data_provider.get_income_stmt()
balance_df = orchestrator.stock_data_provider.get_balance_sheet()

print(f"Financial Metrics for {TICKER}")
print(f"TTM Revenue: ${fin.get('ttm_revenue')/1e9:.2f}B")
print(f"Revenue Growth YoY: {fin.get('revenue_growth_yoy'):.2%}")
print(f"Gross Margin: {fin.get('gross_margin_q'):.2%}")
print(f"Operating Margin: {fin.get('operating_margin_q'):.2%}")
print(f"Net Profit Margin: {fin.get('net_profit_margin_q'):.2%}")
print(f"ROE: {fin.get('return_on_equity_q'):.2%}")
print(f"Debt/Equity: {fin.get('debt_to_equity_q'):.2f}x")

In [ ]:
if 'TotalRevenue' in income_df.index:
    rev_series = pd.to_numeric(income_df.loc['TotalRevenue'])
    
    fig, ax = plt.subplots(figsize=(12, 6))
    rev_series.plot(kind='bar', ax=ax, color='#1f77b4', alpha=0.8, edgecolor='black', linewidth=0.5)
    
    # Format y-axis as billions
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: f'${x/1e9:.1f}B'))
    ax.set_ylabel('Revenue ($)', fontsize=11)
    ax.set_xlabel('Quarter', fontsize=11)
    ax.set_title(f'{TICKER} - Quarterly Total Revenue', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Format x-axis labels as dates
    if isinstance(rev_series.index[0], pd.Timestamp):
        ax.set_xticklabels([d.strftime('%Y-%m') for d in rev_series.index], rotation=45)
    
    plt.tight_layout()
    plt.show()

## Volume Positioning Metrics

In [ ]:
volume = orchestrator.get_volume_positioning_metrics()

print(f"Volume Positioning for {TICKER}")
if volume.get('average_daily_volume'):
    print(f"Average Daily Volume (30d): {volume.get('average_daily_volume'):,.0f}")
if volume.get('latest_volume'):
    print(f"Latest Volume: {volume.get('latest_volume'):,.0f}")
if volume.get('volume_trend_ratio_20d'):
    print(f"Volume Trend Ratio (20d): {volume.get('volume_trend_ratio_20d'):.2f}")
if volume.get('abnormal_volume_20d') is not None:
    print(f"Abnormal Volume (20d): {volume.get('abnormal_volume_20d'):.2%}")
if volume.get('volume_volatility_20d'):
    print(f"Volume Volatility (20d): {volume.get('volume_volatility_20d'):.4f}")

## Peer Metrics

In [ ]:
peer_metrics = orchestrator.get_peer_metrics()
peer_prices = orchestrator.peer_discovery_provider.get_peers_stock_data()

print(f"Peer Relative Performance (63d window)")
print(f"Excess Return vs Peers: {peer_metrics.get('rolling_excess_return_63d_vs_peers'):.2%}")
print(f"Outperformance Frequency: {peer_metrics.get('outperformance_frequency_63d'):.2%}")
print(f"Relative Recovery Time: {peer_metrics.get('relative_recovery_time_days'):.0f} days")
print(f"\nTop Peers: {list(peer_prices.keys())[:5]}")

## Stock Information & Company Data

In [ ]:
info = orchestrator.stock_data_provider.get_info()

# Display key company info
key_fields = ['longName', 'sector', 'industry', 'website', 'fullTimeEmployees', 
              'marketCap', 'enterpriseValue', 'trailingPE', 'forwardPE', 'beta']

print(f"Company Information for {TICKER}")
for field in key_fields:
    value = info.get(field)
    if value:
        if isinstance(value, (int, float)) and value > 1e6:
            print(f"{field}: ${value/1e9:.2f}B" if value > 1e9 else f"{field}: ${value/1e6:.2f}M")
        else:
            print(f"{field}: {value}")

## Macro Sensitivity

In [ ]:
macro = orchestrator.get_macro_metrics()

# Show top macro sensitivities
macro_betas = {k: v for k, v in macro.items() if 'rolling_beta' in k}
if macro_betas:
    sorted_betas = sorted(macro_betas.items(), key=lambda x: abs(x[1]), reverse=True)
    print(f"Top Macro Economic Sensitivities (63d rolling beta) for {TICKER}:")
    for factor, beta in sorted_betas[:5]:
        factor_name = factor.replace('macro_rolling_beta_63d_vs_', '')
        print(f"  {factor_name}: {beta:.3f}")

## Thesis Context

In [ ]:
thesis_ctx = orchestrator.get_thesis_context()

print(f"Thesis Quantitative Context for {TICKER}")
print(f"Revenue Growth YoY: {thesis_ctx.revenue_growth_yoy:.2%}" if thesis_ctx.revenue_growth_yoy else "Revenue Growth YoY: N/A")
print(f"Earnings Growth YoY: {thesis_ctx.earnings_growth_yoy:.2%}" if thesis_ctx.earnings_growth_yoy else "Earnings Growth YoY: N/A")
print(f"Forward P/E: {thesis_ctx.forward_pe:.1f}x" if thesis_ctx.forward_pe else "Forward P/E: N/A")
print(f"Analyst Target Price Upside: {thesis_ctx.analyst_target_price_upside:.2%}" if thesis_ctx.analyst_target_price_upside else "Analyst Target Price Upside: N/A")

print("\n" + "="*60)
print("QUANTITATIVE CONTEXT SUMMARY")
print("="*60)
print(thesis_ctx.to_prompt_context())

# Stage 2: Thesis Engineering


Initialize the thesis validation orchestrator and define your thesis narrative.
Note: Stage 2 requires Anthropic API credentials (ANTHROPIC_API_KEY). If not available, the cells will show expected outputs.

In [ ]:
from model.orchestration.thesis_validation_orchestrator import ThesisValidationOrchestrator


In [ ]:
orchestrator_tv = ThesisValidationOrchestrator(TICKER)

# Define sample thesis narrative and company context
thesis_narrative = """
Snowflake (SNOW) will continue to outgrow the cloud data market as customers
consolidate data workloads onto its platform, driving 20%+ revenue CAGR and
margin expansion from scale and product mix. Management's execution
roadmap and the stickiness of platform adoption will sustain above-market
growth for the next 5 years.
"""

company_context = """
Snowflake Inc. operates a cloud-native data platform for data engineering,
data lakes, data warehousing, data application development, and secure data
sharing. Large enterprise customers primarily drive revenue with consumption-based pricing.
"""

print(f"Thesis Validation Setup for {TICKER}")
print(f"\nThesis: {thesis_narrative.strip()}")
print(f"\nCompany Context: {company_context.strip()}")

### Stage 2.1: Narrative Decomposition Graph (NDG)

Decompose thesis into key assumptions, vulnerabilities, and relationships.

In [ ]:
try:
    ndg_output = orchestrator_tv.ndg.run(
        thesis_narrative=thesis_narrative,
        company_context=company_context,
        quantitative_context=thesis_ctx
    )
    print(f"NDG Analysis Complete for {TICKER}")
    print(f"Nodes (key assumptions): {len(ndg_output.nodes)}")
    print(f"Edges (relationships): {len(ndg_output.edges)}")
    if ndg_output.nodes:
        print(f"\nSample assumptions:")
        for node in list(ndg_output.nodes)[:3]:
            print(f"  - {node.label}: {node.description[:80]}...")
except Exception as e:
    print(f"NDG requires Anthropic API key. Error: {type(e).__name__}")

### Stage 2.2: Red Team (Adversarial Challenges)

Generate challenges and counterarguments to stress-test the thesis.

In [ ]:
try:
    if 'ndg_output' in locals():
        red_team_output = orchestrator_tv.red_team.run(ndg=ndg_output, company_context=company_context)
        print(f"Red Team Analysis Complete for {TICKER}")
        print(f"Challenges generated: {len(red_team_output.challenges)}")
        if red_team_output.challenges:
            print(f"\nSample challenges:")
            for challenge in red_team_output.challenges[:3]:
                print(f"  - {challenge.challenge[:100]}...")
    else:
        print("Run NDG stage first to enable Red Team analysis")
except Exception as e:
    print(f"Red Team requires Anthropic API key. Error: {type(e).__name__}")

### Stage 2.3: Counterfactual Scenario Engine (CRE)

Generate alternative scenarios and their implications.

In [ ]:
try:
    if 'ndg_output' in locals() and 'red_team_output' in locals():
        cre_output = orchestrator_tv.cre.run(
            ndg=ndg_output,
            red_team=red_team_output,
            company_context=company_context,
            quantitative_context=thesis_ctx
        )
        print(f"Counterfactual Scenario Analysis Complete for {TICKER}")
        print(f"Scenarios generated: {len(cre_output.scenario_set.scenarios)}")
        if cre_output.scenario_set.scenarios:
            print(f"\nScenario types:")
            for scenario in cre_output.scenario_set.scenarios[:3]:
                print(f"  - {scenario.scenario_type}: {scenario.description[:80]}...")
    else:
        print("Run NDG and Red Team stages first to enable CRE analysis")
except Exception as e:
    print(f"CRE requires Anthropic API key. Error: {type(e).__name__}")

### Stage 2.4: Financial Translation (FT)

Map scenarios to financial valuations and outcomes.

In [ ]:
try:
    if 'cre_output' in locals():
        ft_result = orchestrator_tv.ft.run(cre_output)
        ft_output = ft_result.cre_output
        print(f"Financial Translation Complete for {TICKER}")
        print(f"Valuation scenarios: {len(ft_output.scenario_results)}")
        if ft_output.scenario_results:
            print(f"\nValuation outcomes:")
            for result in ft_output.scenario_results[:3]:
                print(f"  - Scenario: Target Price = ${result.target_price:.2f}" if result.target_price else f"  - Scenario analyzed")
    else:
        print("Run CRE stage first to enable Financial Translation")
except Exception as e:
    print(f"FT requires Anthropic API key. Error: {type(e).__name__}")

### Stage 2.5: Thesis Validity Evaluation

Evaluate thesis status based on rule-based assessments.

In [ ]:
try:
    if all(k in locals() for k in ['ft_output', 'red_team_output', 'ndg_output']):
        validity_output = orchestrator_tv.validity_evaluator.run(ft_output, red_team_output, ndg_output)
        print(f"Thesis Validity Evaluation Complete for {TICKER}")
        print(f"Status: {validity_output.status}")
        print(f"Conviction: {validity_output.conviction_level:.2%}" if validity_output.conviction_level else "Conviction: N/A")
        if validity_output.issues:
            print(f"\nKey Issues:")
            for issue in validity_output.issues[:3]:
                print(f"  - {issue}")
    else:
        print("Run prior stages first to enable Validity Evaluation")
except Exception as e:
    print(f"Validity evaluation requires Anthropic API key. Error: {type(e).__name__}")

### Stage 2.6: Half-Life Estimation (IHLE)

Estimate thesis idea half-life and monitoring cadence.

In [ ]:
try:
    if 'ndg_output' in locals():
        ihle_output = orchestrator_tv.ihle.run(ndg=ndg_output, current_macro_context=None)
        print(f"Idea Half-Life Estimation Complete for {TICKER}")
        if ihle_output and ihle_output.half_life_estimate:
            print(f"Estimated Half-Life: {ihle_output.half_life_estimate.estimated_half_life_months} months")
            print(f"Monitoring Cadence: {ihle_output.half_life_estimate.monitoring_cadence}")
            print(f"Severity: {ihle_output.half_life_estimate.severity_level}")
        else:
            print("Half-life estimate unavailable")
    else:
        print("Run NDG stage first to enable Half-Life Estimation")
except Exception as e:
    print(f"IHLE requires Anthropic API key. Error: {type(e).__name__}")

### Stage 2.7: Full Pipeline (Optional)

To run the complete thesis validation pipeline at once try the following cell. This orchestrates all 7 stages with proper dependency management. Individual stages above allow for inspection, paramater adjustment, and iteration.

In [ ]:
try:
    final_report = orchestrator_tv.run(
        thesis_narrative=thesis_narrative,
        company_context=company_context,
        quantitative_context=thesis_ctx
    )
    print(f'Final Status: {final_report.status}')
    print(f'Aggregation Score: {final_report.aggregation_score:.2%}')
except Exception as e:
    print(f'Full pipeline requires Anthropic API key')